In [0]:
import tensorflow as tf
import numpy as np

In [0]:
from google.colab import drive
drive.mount('/content/drive')

# Sequence-to-Sequence (seq2seq)

seq2seq: 주로 기계번역에 많이 사용된다.
* 입력을 위한 신경망인 인코더, 인코더와 디코더 사이의 컨텍스트 벡서, 출력을 위한 신경망인 디코더로 구성되어있다.
* 트레이닝 과정과 테스트 과정이 다르다

# Seq2Seq 단어 번역기 (tensorflow 버전)

영어 단어를 한국어 단어로 번역

In [0]:
import tensorflow as tf
import numpy as np

S: 디코딩 입력의 시작

E: 디코딩 출력의 끝

P: 패딩, time step 크기보다 작은 단어의 경우, 빈 자리를 채우는 문자
* time step 4인 경우
  * love -> l, o, v, e
  * my -> m, y, P, P


## 데이터 준비

In [2]:
char_arr = [c for c in "SEPabcdefghijklmnopqrstuvwxyz단어나무놀이소녀목록사랑"]
char_arr    # corpus

['S',
 'E',
 'P',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z',
 '단',
 '어',
 '나',
 '무',
 '놀',
 '이',
 '소',
 '녀',
 '목',
 '록',
 '사',
 '랑']

In [3]:
num_dic = {n:i for i,n in enumerate(char_arr)}
num_dic    # 딕셔너리

{'E': 1,
 'P': 2,
 'S': 0,
 'a': 3,
 'b': 4,
 'c': 5,
 'd': 6,
 'e': 7,
 'f': 8,
 'g': 9,
 'h': 10,
 'i': 11,
 'j': 12,
 'k': 13,
 'l': 14,
 'm': 15,
 'n': 16,
 'o': 17,
 'p': 18,
 'q': 19,
 'r': 20,
 's': 21,
 't': 22,
 'u': 23,
 'v': 24,
 'w': 25,
 'x': 26,
 'y': 27,
 'z': 28,
 '나': 31,
 '녀': 36,
 '놀': 33,
 '단': 29,
 '랑': 40,
 '록': 38,
 '목': 37,
 '무': 32,
 '사': 39,
 '소': 35,
 '어': 30,
 '이': 34}

In [5]:
dic_len = len(num_dic)    # 41
dic_len

41

In [0]:
# 영어 단어를 한글로 번역하기 위한 학습용 데이터

seq_data = [["word", "단어"], ["tree", "나무"], ["game", "놀이"], ["girl", "소녀"], ["list", "목록"], ["love", "사랑"]]

## 참고: np.eye() 로 원핫 인코딩

In [33]:
np.eye(3)

array([[1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.]])

In [34]:
np.eye(dic_len)    # 딕셔너리 길이만큼의 행이 생김 -> 딕셔너리 전체를 원핫인코딩 한것과 같은 형태

array([[1., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 0., 1.]])

In [36]:
np.eye(dic_len)[[1, 5, 9]]    # np.eye()의 1, 5, 9번째 행 -> 딕셔너리의 1, 5, 9번째 인덱스넘버를 원핫 인코딩한 형태

array([[0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0.]])

## 함수 만들기 1

In [0]:
def make_batch(seq_data):    # ("word", "PPPP")
  input_batch = []
  output_batch = []
  target_batch = []
  for seq in seq_data:
    input = [num_dic[n] for n in seq[0]]    # 인코더 셀의 입력값
    output = [num_dic[n] for n in ("S" + seq[1])]    # 디코더 셀의 입력값    # "S"는 시작을 나타낸다
    target = [num_dic[n] for n in (seq[1] + "E")]    # 디코더 셀의 출력값    # "E"는 끝을 나타낸다
    input_batch.append(np.eye(dic_len)[input])    # 원핫 인코딩
    output_batch.append(np.eye(dic_len)[output])    # 원핫 인코딩
    target_batch.append(target)    # 출력값만 원핫 인코딩이 아니다
    # 출력값이 원핫인코딩 되어있지 않을 때 쓰는 softmax 함수 = sparse_softmax_cross_entropy_with_logits
  return input_batch, output_batch, target_batch

## 파라미터 설정

In [0]:
learning_rate = 0.01
n_hidden = 128
total_epoch = 100
# 입력과 출력의 형태는 원핫 인코딩으로 크기가 같다
n_class = n_input = dic_len

## 신경망 구성 및 학습


In [30]:
# 신경망 구성

tf.reset_default_graph()

# seq2seq 모델은 인코더의 입력과 디코더의 입력의 형식이 같다: [batch size, time step, input size]
# 디코더의 출력 형식: [batch size, time step]

enc_input = tf.placeholder(tf.float32, [None, None, n_input])    # batch size와 글자 길이(time step)가 정해져 있지 않다
# 같은 배치인 경우에는 입력 데이터의 글자수 (time step) 는 모두 같아야 한다
dec_input = tf.placeholder(tf.float32, [None, None, n_input])
targets = tf.placeholder(tf.int64, [None, None])

with tf.variable_scope("encode"):
  # 인코더 셀 구성
  enc_cell = tf.nn.rnn_cell.BasicRNNCell(n_hidden)
  # 오버피팅 방지하기 위해 드랍아웃
  enc_cell = tf.nn.rnn_cell.DropoutWrapper(enc_cell, output_keep_prob=0.5)
  # 드라이버 설정
  outputs, enc_states = tf.nn.dynamic_rnn(enc_cell, enc_input, dtype=tf.float32)

with tf.variable_scope("decode"):
  # 디코더 셀 구성
  dec_cell = tf.nn.rnn_cell.BasicRNNCell(n_hidden)
  dec_cell = tf.nn.rnn_cell.DropoutWrapper(dec_cell, output_keep_prob=0.5)
  # 드라이버 설정
  outputs, dec_states = tf.nn.dynamic_rnn(dec_cell, dec_input, initial_state=enc_states, dtype=tf.float32)
  # initial_state=enc_states: seq2seq 모델은 인코더 셀의 최종 상태 값을 디코더 셀의 초기 상태값으로 입력해줘야 한다

model = tf.layers.dense(outputs, n_class, activation=None)
# model 수행 결과가 [batch size, time step, input]
cost = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits=model, labels=targets))
optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)



# 학습

sess = tf.Session()
sess.run(tf.global_variables_initializer())

input_batch, output_batch, target_batch = make_batch(seq_data)

for epoch in range(total_epoch):
  _, loss = sess.run([optimizer, cost], feed_dict={enc_input:input_batch, dec_input: output_batch, targets: target_batch})
  print("epoch: {:04d}, cost: {:.6f}".format(epoch+1, loss))
print("모델 완료")

epoch: 0001, cost: 3.786503
epoch: 0002, cost: 2.797024
epoch: 0003, cost: 1.622213
epoch: 0004, cost: 0.990022
epoch: 0005, cost: 0.603112
epoch: 0006, cost: 0.285041
epoch: 0007, cost: 0.148995
epoch: 0008, cost: 0.089851
epoch: 0009, cost: 0.047263
epoch: 0010, cost: 0.060495
epoch: 0011, cost: 0.029462
epoch: 0012, cost: 0.036320
epoch: 0013, cost: 0.034499
epoch: 0014, cost: 0.015579
epoch: 0015, cost: 0.005542
epoch: 0016, cost: 0.009919
epoch: 0017, cost: 0.005139
epoch: 0018, cost: 0.003253
epoch: 0019, cost: 0.005737
epoch: 0020, cost: 0.005223
epoch: 0021, cost: 0.005035
epoch: 0022, cost: 0.001811
epoch: 0023, cost: 0.001322
epoch: 0024, cost: 0.004328
epoch: 0025, cost: 0.003073
epoch: 0026, cost: 0.001941
epoch: 0027, cost: 0.008183
epoch: 0028, cost: 0.000624
epoch: 0029, cost: 0.010115
epoch: 0030, cost: 0.000757
epoch: 0031, cost: 0.007256
epoch: 0032, cost: 0.000576
epoch: 0033, cost: 0.001436
epoch: 0034, cost: 0.000331
epoch: 0035, cost: 0.001294
epoch: 0036, cost: 0

## 함수 만들기 2

In [0]:
# 단어를 입력받아서 번역된 단어를 예측하고 디코딩하는 함수

def translate(word):
  # 단어 번역
  # [영어 단어, 한글 단어]
  # 예측시에는 한글단어를 모르므로, 디코더의 입출력값을 의미없는 값인 P로 채운다: ['word', 'PPPP']
  seq_data = [word, "P"*len(word)]
  input_batch, output_batch, target_batch = make_batch([seq_data])
  # [batch size, time step, input]
  prediction = tf.argmax(model, 2)    # model 수행 결과가 [batch size, time step, input]
  # 2번째 차원인 input 차원을 argmax로 해서 가장 확률이 높은 글자를 예측값으로 한다.
  result = sess.run(prediction, feed_dict={enc_input:input_batch, dec_input: output_batch, targets: target_batch})
  decoded = [char_arr[i] for i in result[0]]    # 결과값이 숫자로 나오니까 숫자의 인덱스에 해당하는 문자를 가져온 것
  # 출력의 끝에 해당하는 "E" 이후의 글자들을 제거하고 문자열을 출력
  end = decoded.index("E")    # "E"라는 글자의 인덱스 넘버를 가져와서 end에 넣어라
  translated = "".join(decoded[:end])
  return translated

## 번역

In [32]:
print("word -> ", translate("word"))    # 단어
print("wodr -> ", translate("wodr"))    # 단어
print("love -> ", translate("love"))    # 사랑
print("loev -> ", translate("loev"))    # 사랑
print("abcd -> ", translate("abcd"))    # ????

word ->  단어
wodr ->  단어
love ->  사랑
loev ->  사랑
abcd ->  무사랑
